<a href="https://colab.research.google.com/github/1306minyoung/data/blob/main/7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

심층신경망

- 2개이상의 츠을 포함한 신경망
- 종종 다층인공신경망, 심층신경망, 딥러닝을 같은 의미로 사용

In [1]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
from sklearn.model_selection import train_test_split
train_scaled = train_input/255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

**은닉층**

입력층과 출력층 사이에 있느 모든 층

출력층에서는 소프트맥스나 시그모이드 함수를 써야함

그러나 은닉층은 활성화 함수의 선택이 자유로움

**왜 은닉층에 활성화 함수를 적용?**

활성화 함수 없이 은닉층만 있다면 의미가 없기 때문

은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈.

선형 계산을 적당하게 비선형적으로 비틀어 주어야 한다. 그래야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있는 것.

In [5]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
#dense1: 은닉층, 100개의 뉴런을 가진 밀집층. 활성화 함수를 시그모이드로 지정하고 입력의 크기는(784,)로 지정

In [6]:
dense2 = keras.layers.Dense(10, activation='softmax')
#dense2: 출력층. 10개의 클래스를 분류하므로 10개의 뉴런을 둠. 활성화 함수는 소프트맥스.

In [7]:
model = keras.Sequential([dense1, dense2])

summary() : 층에 대한 유용한 정보 얻을 수 있음

param 부분: 픽셀784+1개가 은닉층 100개와 연결되니까 785*100이라서 78500, 은닉층 뉴런 100개+1개가 10개의 출력층 뉴런과 모두 연결되니까 101*10이라서 1010
(1개씩 왜 더하냐? -> 뉴런마다 한개씩 절편이 있어서)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


##층을 추가하는 방법

따로 Dense 클래스를 추가하지 않고 Sequential클래스의 생성자 안에서 바로 dense 클래스의 객체를 만듦

In [10]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


편리하지만 아주 많은 층을 추가하려면 Sequential 클래스의 생성자가 매우 길어지고 조건에 따라 층을 추가할 수 없음

In [12]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(None, 100)인 이유. 첫번째 차원은 샘플의 개수를 나타내는데, 샘플개수가 아직 정의되어 있지 않기 때문에 그런것!

In [13]:
#모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 11s 6ms/step - loss: 0.5687 - accuracy: 0.8062
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4116 - accuracy: 0.8512
Epoch 3/5
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3762 - accuracy: 0.8636
Epoch 4/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3536 - accuracy: 0.8725
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3349 - accuracy: 0.8787


In [14]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3581 - accuracy: 0.8692


[0.3580542802810669, 0.8691666722297668]

##렐루 함수

활성함수를 바꿔보자

초창기 인공신경망의 은닉층에 많이 사용된 함수는 시그모이드.

오른쪽과 왼쪽 끝으로 갈 수록 그래프가 누워있으니까 올바른 출력을 만드는데 신속 대응 불가.

특히 층이 많은 경우 심층 신경망인 경우 그 효과가 누적되어 더 어려움.

그래서 이를 개선 하기 위해 다른 종류의 활성화 함수가 제안됨. -> 렐루함수!

렐루함수는 입력이 양수일 때 입력을 그대로 통과시키고 음수일 경우 0으로 설정.

렐루함수는 max(0,z)와 같이 쓸수 있음


이 함수는 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력.

렐루는 특히 이미지 출력에서 좋은 성능을 냄.

**Flatten층**

배치 차원 제외 나머지 입력차원을 모두 일렬로 펼치는 역할만 함.

데이터의 첫번째축을 배치축 또는 배치 차원이라고 부름

입력에 곱해지는 가중치나 절편이 없어 성능에 기여하는 것은 없음

그러나 flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라 부름

In [15]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


첫번째 등장하는 flatten 클래스에 포함된 모델 파라미터는 0개

케라스의 flatten 층을 신경망 모델에 추가하면 입력 값의 차원 짐작 가능.

케라스 API는 입력 데이터에 대한 전처리 과정을 가능하면 모델에 포함시킴

In [17]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [18]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5390 - accuracy: 0.8088
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3939 - accuracy: 0.8584
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3551 - accuracy: 0.8724
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3303 - accuracy: 0.8811
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3164 - accuracy: 0.8885


**옵티마이저**

신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법

사용할 경사 하강법 알고리즘과 그 파라미터를 지정하는 매개변수

신경망에는 하이퍼파라미터가 많음

추가할 은닉층의 개수, 은닉층의 뉴런개수, 활성화 함수, 층의 종류 등

케라스는 다양한 종류의 경사 하강법 알고리즘을 제공해 이들을 옵티마이저라고 함

**모멘텀 최적화**

모멘텀(관성, 운동량) 조절

sgd=keras.optimizers.SGD(momentum=0.9)#기본값 0

새로운 가중치와 절편 계산시 과거의 가중치와 절편의 변화량을 어느정도 반영할거지 결정

**네스테로프 모멘텀 최적화**

모멘텀 최적화 2번 반복

손실함수의 입력인 이전의 기울기 절편에 추가로 이전 속도가 더해진다.

In [19]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')
sgd = keras.optimizers.SGD(learning_rate=0.1)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [20]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5212 - accuracy: 0.8191
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3929 - accuracy: 0.8595
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3547 - accuracy: 0.8718
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3259 - accuracy: 0.8798
Epoch 5/5
375/375 [==============================] - 1s 2ms/step - loss: 0.3407 - accuracy: 0.8782


[0.340666264295578, 0.878166675567627]

Adam : 모멘텀 최적화와 RMSprop의 장점을 접목한 것.

keras.optimizers 패키지 아래에 있고, learning_rate 매개변수 기본값은 0.001임.

모멘텀 최적화에 있는 그래디언트의 지수 감소 평균을 조절하기 위해 beta_1 매개변수가 있음(기본값 0.9)

RMSprop에 있는 그레디언트 제곱의 지수감소 평균을 조절하기 위해 beta_2 매개변수가 있음(기본값 0.999)